# Cómo usar `control`

In [ ]:
import control
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (15, 5)

Vamos a trabajar con el sistema isto en el video 3 (no hace falta ver el video para seguir este tutorial) que tiene la siguiente función de transferencia

$$
G = \frac{1}{(s+3)(s^2+1.4s+100)}
$$

En `control` existe la clase `TransferFunction` la que nos permite escribir una función de transferencia dado el numerador y el denominador en numpy arrays (o listas), donde los coeficientes son $[a_0s^n, ... , a_{n-1}s^0]$ así por ejemplo, el polinomio $3s^3 + s + 40$ iene dado por un array cuyo contenido es $[3, 0, 1, 40]$.

Los objetos que representan las funciones de transferencia aceptan multiplicaciones, por lo que si tenemos las variables `tf1` y `tf2`, ambas funciones de transferencia, podemos simplemente multiplicarlas con el operador `*`. Vamos a aplicar esto notando que $G$ está compuesta de dos funciones de transferencia multiplicadas $G_1$ y $G_2$.


$$
\begin{align}
    G_1 &= \frac{1}{s+3}\\
    G_2 &= \frac{1}{s^2+1.4s+100}
\end{align}
$$

A continuación se implementa esto en python

In [ ]:
G_1 = control.TransferFunction(1, [1, 3])
G_2 = control.TransferFunction(1, [1, 1.4, 100])
G = G_1 * G_2

G

Podemos ver que la representación de las funciones de transferencia queda bonita ❤️


`control` nos permite poder ver fácilmente la respuesta al escalón utilizando `control.step_response`


In [ ]:
t, y = control.step_response(G)

plt.figure()
plt.grid()
plt.xlabel("Tiempo [s]")
plt.ylabel("respuesta al escalón y(t)")
plt.plot(t, y);

Hasta ahora hemos isto la función de transferencia en __lazo abierto__, si queremos ver la respuesta en lazo cerrado tenemos que poner un controlador, veamos el caso del controlador PID:


La nueva función de transferencia de lazo cerrado tiene la forma 

$$
TF = \frac{\{\text{PID}\} G}{1 + \{\text{PID}\} G}
$$

donde $\{PID\}$ corresponde a la función de transferencia del bloque $\text{PID}$

El bloque PID es 

$$
\text{PID} = K_p + \frac{K_i}{s} + K_d s
$$


Así podemos crear una clase PID que representa este bloque

In [ ]:
class PID(control.TransferFunction):
    def __init__(self, k_p, k_i, k_d):
        """
        Clase que representa el bloque PID a la que hay que entregar
        k_p, k_i y k_d que son las constantes de proporción proporcional, 
        integral y diferencial respectivamente.
        """
        p = control.TransferFunction([k_p], [1])
        i = control.TransferFunction([k_i], [1, 0])
        d = control.TransferFunction([k_d, 0], [1])
        pid = p + i + d
        
        super().__init__(pid.num[0][0], pid.den[0][0])

Además vamos a hacer una función que dada una función de transferencia, nos devuela la función de transferencia en lazo cerrado

In [ ]:
def closed_loop(tf):
    return tf / (1 + tf)

Veamos cómo es la función de lazo cerrado con solo controlador proporcional con $K_p=1$

In [ ]:
pid = PID(1, 0, 0)
tf = closed_loop(pid * G)
t, y = control.step_response(tf)

# plot
plt.figure()
plt.plot(t, y)
plt.xlabel("Tiempo [s]")
plt.ylabel("Respuesta al escalón y(t)")
plt.grid()
plt.show()

Notar que no estamos ni cerca del valor deseado, que eneste caso es 1. Probaremos distintas configuraciones del control PID para lograr esto. Pero antes veamos los polos del sistema que hemos creado con la función `control.pzmap` 

In [ ]:
poles, zeros = control.pzmap(tf)

Vemos que nos devuelve los polos y los ceros además de hacer un gráfico con los mismos

Probemos nuevos valores para el control PID y veamos las respuestas en el tiempo y en los ceros y polos

## $K_p=25, K_i = 1000, K_d = 0$

In [ ]:
pid = PID(25, 1000, 0)
tf = closed_loop(pid * G)
t, y = control.step_response(tf)

# plot
plt.figure()
plt.plot(t, y)
plt.xlabel("Tiempo [s]")
plt.ylabel("Respuesta al escalón y(t)")
plt.grid()
plt.show()

In [ ]:
poles, zeros = control.pzmap(tf)

## $K_p=25, K_i = 1000, K_d = 20$

In [ ]:
pid = PID(25, 1000, 20)
tf = closed_loop(pid * G)
t, y = control.step_response(tf)

# plot
plt.figure()
plt.plot(t, y)
plt.xlabel("Tiempo [s]")
plt.ylabel("Respuesta al escalón y(t)")
plt.grid()
plt.show()

In [ ]:
poles, zeros = control.pzmap(tf)